## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Nov. 4th Nov. 11th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,November-5-2024,Condo/Co-op,1221 SW 122nd Ave #303,Miami,FL,33184.0,270000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1221-SW-122nd-...,MARMLS,A11660561,N,Y,25.756864,-80.390292


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
39,PAST SALE,2024-11-08,Condo/Co-op,19380 Collins Ave #914,Sunny Isles Beach,FL,33160.0,380000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/19...,MARMLS,A11656166,N,Y,25.954860,-80.123290
11,PAST SALE,2024-11-08,Condo/Co-op,15545 N Miami Lakeway N Unit 202-21,Miami Lakes,FL,33014.0,400000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Lakes/15545-Mi...,MARMLS,A11658091,N,Y,25.917276,-80.301521
23,PAST SALE,2024-11-08,Condo/Co-op,20355 NE 34th Ct #2428,Aventura,FL,33180.0,415000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20355-NE-34th-...,MARMLS,A11646582,N,Y,25.964569,-80.132758
63,PAST SALE,2024-11-08,Condo/Co-op,10 Aragon Ave #807,Coral Gables,FL,33134.0,725000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/10-Arag...,Beaches MLS,F10447393,N,Y,25.750306,-80.256031
40,PAST SALE,2024-11-08,Condo/Co-op,2775 W 62nd Pl #207,Hialeah,FL,33016.0,290000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/2775-W-62nd-...,MARMLS,A11653965,N,Y,25.879318,-80.339200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,PAST SALE,2024-11-04,Condo/Co-op,6600 W 2nd Ct #11,Hialeah,FL,33012.0,230000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/6600-W-2nd-C...,MARMLS,A11646911,N,Y,25.892643,-80.344729
55,PAST SALE,2024-11-04,Condo/Co-op,1010 Brickell Ave #1602,Miami,FL,33131.0,885000.0,2.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1010-Brickell-...,MARMLS,A11637955,N,Y,25.764203,-80.192181
44,PAST SALE,2024-11-04,Condo/Co-op,8620 NW 97th Ave #105,Doral,FL,33178.0,410000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Doral/8620-NW-97th-A...,MARMLS,A11648493,N,Y,25.852125,-80.355045
52,PAST SALE,2024-11-04,Condo/Co-op,2655 S Bayshore Dr #1008,Coconut Grove,FL,33133.0,940000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,MARMLS,A11634175,N,Y,25.731196,-80.236270


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [66]:
# import requests

# url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

# querystring = {"url":"https://www.redfin.com/CT/New-Haven/Pierpont/apartment/176060870"}

# headers = {"x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61","x-rapidapi-host": "redfin-com-data.p.rapidapi.com"}

# response = requests.get(url, headers=headers, params=querystring)

# print(response.json())

{'data': None, 'errors': 'Error while extracting response for type [class com.fasterxml.jackson.databind.JsonNode] and content type [application/json]; nested exception is org.springframework.http.converter.HttpMessageNotReadableException: JSON parse error: Unexpected end-of-input within/between Object entries; nested exception is com.fasterxml.jackson.core.JsonParseException: Unexpected end-of-input within/between Object entries\n at [Source: (org.springframework.util.StreamUtils$NonClosingInputStream); line: 1, column: 135435]', 'status': False, 'message': 'Errors'}


In [13]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

        print(response.status_code)
        print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [14]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

200
https://www.redfin.com/FL/Miami/2627-S-Bayshore-Dr-33133/unit-1901/home/43343001
200
https://www.redfin.com/FL/Miami-Beach/100-S-Pointe-Dr-33139/unit-1507/home/43282097
200
https://www.redfin.com/FL/Key-Biscayne/201-Crandon-Blvd-33149/unit-900/home/42910968
200
https://www.redfin.com/FL/Sunny-Isles-Beach/18201-Collins-Ave-33160/unit-4004/home/42697261
200
https://www.redfin.com/FL/Key-Biscayne/881-Ocean-Dr-33149/unit-18E/home/42924246
200
https://www.redfin.com/FL/Miami/20201-E-Country-Club-Dr-33180/unit-1105/home/43449407
200
https://www.redfin.com/FL/Miami-Beach/10-Venetian-Way-33139/unit-1103/home/43275846
200
https://www.redfin.com/FL/Coral-Gables/111-Edgewater-Dr-33133/unit-2A/home/43415790
200
https://www.redfin.com/FL/Coral-Gables/13645-Deering-Bay-Dr-33158/unit-154/home/42815629
200
https://www.redfin.com/FL/Miami/3266-Virginia-St-33133/unit-B3/home/42755414


In [15]:
response_df = pd.DataFrame(response_list)

In [16]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [17]:
df_top_ten = merged_df

## Current Week's Values

In [18]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

83


In [19]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$735,595


In [20]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$557


In [21]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$61,054,385


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
120
Input Previous Week Condo Average Sales Price:
770797
Input Previous Week Condo Average PSF:
543
Input Previous Week Condo Sales total (ex: 198_000_000)
92_500_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-05,Condo/Co-op,2627 S Bayshore Dr #1901,Miami,FL,33133.0,5400000.0,5.0,5.5,GROVENOR HOUSE CONDO,4026.0,NaN,2005.0,NaN,1341.0,5859.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11642744,N,Y,25.732211,-80.234936,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,None,None,None,None,None,None,None,None,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-05,Condo/Co-op,2627 S Bayshore Dr #1901,Miami,FL,33133.0,5400000.0,5.0,5.5,GROVENOR HOUSE CONDO,4026.0,NaN,2005.0,NaN,1341.0,5859.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11642744,N,Y,25.732211,-80.234936,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,None,None,None,None,None,None,None,None,1,orange
1,PAST SALE,2024-11-05,Condo/Co-op,100 S Pointe Dr #1507,Miami Beach,FL,33139.0,5000000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1870.0,NaN,2002.0,NaN,2674.0,4417.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11501307,N,Y,25.767161,-80.132776,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,None,None,None,None,None,None,None,None,2,blue
2,PAST SALE,2024-11-04,Condo/Co-op,201 Crandon Blvd #900,Key Biscayne,FL,33149.0,2550000.0,3.0,3.0,KEY COLONY NO 1,2169.0,NaN,1978.0,NaN,1176.0,2323.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,MARMLS,A11653306,N,Y,25.696814,-80.158094,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,None,None,None,None,None,None,None,None,3,blue
3,PAST SALE,2024-11-05,Condo/Co-op,18201 Collins Ave #4004,Sunny Isles Beach,FL,33160.0,2300000.0,3.0,3.5,TRUMP ROYALE CONDO,2174.0,NaN,2008.0,NaN,1058.0,5235.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11584543,N,Y,25.944933,-80.120001,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,None,None,None,None,None,None,None,None,4,blue
4,PAST SALE,2024-11-05,Condo/Co-op,881 Ocean Dr Unit 18E,Key Biscayne,FL,33149.0,1900000.0,3.0,2.0,CASA DEL MAR CONDO,1875.0,NaN,1971.0,NaN,1013.0,1661.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,MARMLS,A11585591,N,Y,25.685659,-80.157201,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,None,None,None,None,None,None,None,None,5,blue
5,PAST SALE,2024-11-06,Condo/Co-op,20201 E Country Club Dr #1105,Aventura,FL,33180.0,1850000.0,3.0,4.0,HAMPTONS SOUTH CONDO,3280.0,NaN,2004.0,NaN,564.0,2800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20201-E-Countr...,MARMLS,A11620826,N,Y,25.964687,-80.125368,https://www.redfin.com/FL/Miami/20201-E-Countr...,None,None,None,None,None,None,None,None,6,blue
6,PAST SALE,2024-11-04,Condo/Co-op,10 Venetian Way #1103,Miami Beach,FL,33139.0,1850000.0,2.0,2.5,THE GRAND VENETIAN CONDO,1463.0,NaN,2001.0,NaN,1265.0,1794.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/10-Venet...,MARMLS,A11635934,N,Y,25.791150,-80.146649,https://www.redfin.com/FL/Miami-Beach/10-Venet...,None,None,None,None,None,None,None,None,7,blue
7,PAST SALE,2024-11-06,Condo/Co-op,111 Edgewater Dr Unit 2A,Coral Gables,FL,33133.0,1700000.0,4.0,4.0,DAKOTA CONDO,2510.0,NaN,1992.0,NaN,677.0,1111.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/111-Edg...,MARMLS,A11630507,N,Y,25.707219,-80.256084,https://www.redfin.com/FL/Coral-Gables/111-Edg...,None,None,None,None,None,None,None,None,8,blue
8,PAST SALE,2024-11-07,Condo/Co-op,13645 Deering Bay Dr #154,Coral Gables,FL,33158.0,1617946.0,3.0,3.5,DEERING BAY CONDO I,3140.0,NaN,1993.0,NaN,515.0,2894.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/13645-D...,MARMLS,A11628250,N,Y,25.634885,-80.291061,https://www.redfin.com/FL/Coral-Gables/13645-D...,None,None,None,None,None,None,None,None,9,blue
9,PAST SALE,2024-11-04,Condo/Co-op,3266 Virginia St Unit B-3,Miami,FL,33133.0,1400000.0,2.0,3.0,IPANEMA VILLAS CONDO,1620.0,NaN,1993.0,NaN,864.0,1000.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3266-Virginia-...,MARMLS,A11659541,N,Y,25.730508,-80.241884,https://www.redfin.com/FL/Miami/3266-Virginia-...,None,None,None,

In [69]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [70]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-05,Condo/Co-op,2627 S Bayshore Dr #1901,Miami,FL,33133.0,5400000.0,5.0,5.5,GROVENOR HOUSE CONDO,4026.0,NaN,2005.0,NaN,1341.0,5859.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11642744,N,Y,25.732211,-80.234936,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,None,None,None,None,None,None,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_11112024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'GROVENOR HOUSE CONDO at 2627 S Bayshore Dr #1901 in Miami'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-05,Condo/Co-op,2627 S Bayshore Dr #1901,Miami,FL,33133.0,5400000.0,5.0,5.5,GROVENOR HOUSE CONDO,4026.0,NaN,2005.0,NaN,1341.0,5859.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11642744,N,Y,25.732211,-80.234936,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,None,None,None,None,None,None,None,None,1,orange,GROVENOR HOUSE CONDO at 2627 S Bayshore Dr #19...
1,PAST SALE,2024-11-05,Condo/Co-op,100 S Pointe Dr #1507,Miami Beach,FL,33139.0,5000000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1870.0,NaN,2002.0,NaN,2674.0,4417.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11501307,N,Y,25.767161,-80.132776,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,None,None,None,None,None,None,None,None,2,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #1...
2,PAST SALE,2024-11-04,Condo/Co-op,201 Crandon Blvd #900,Key Biscayne,FL,33149.0,2550000.0,3.0,3.0,KEY COLONY NO 1,2169.0,NaN,1978.0,NaN,1176.0,2323.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,MARMLS,A11653306,N,Y,25.696814,-80.158094,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,None,None,None,None,None,None,None,None,3,blue,KEY COLONY NO 1 at 201 Crandon Blvd #900 in Ke...
3,PAST SALE,2024-11-05,Condo/Co-op,18201 Collins Ave #4004,Sunny Isles Beach,FL,33160.0,2300000.0,3.0,3.5,TRUMP ROYALE CONDO,2174.0,NaN,2008.0,NaN,1058.0,5235.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11584543,N,Y,25.944933,-80.120001,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,None,None,None,None,None,None,None,None,4,blue,TRUMP ROYALE CONDO at 18201 Collins Ave #4004 ...
4,PAST SALE,2024-11-05,Condo/Co-op,881 Ocean Dr Unit 18E,Key Biscayne,FL,33149.0,1900000.0,3.0,2.0,CASA DEL MAR CONDO,1875.0,NaN,1971.0,NaN,1013.0,1661.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,MARMLS,A11585591,N,Y,25.685659,-80.157201,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,None,None,None,None,None,None,None,None,5,blue,CASA DEL MAR CONDO at 881 Ocean Dr Unit 18E in...
5,PAST SALE,2024-11-06,Condo/Co-op,20201 E Country Club Dr #1105,Aventura,FL,33180.0,1850000.0,3.0,4.0,HAMPTONS SOUTH CONDO,3280.0,NaN,2004.0,NaN,564.0,2800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20201-E-Countr...,MARMLS,A11620826,N,Y,25.964687,-80.125368,https://www.redfin.com/FL/Miami/20201-E-Countr...,None,None,None,None,None,None,None,None,6,blue,HAMPTONS SOUTH CONDO at 20201 E Country Club D...
6,PAST SALE,2024-11-04,Condo/Co-op,10 Venetian Way #1103,Miami Beach,FL,33139.0,1850000.0,2.0,2.5,THE GRAND VENETIAN CONDO,1463.0,NaN,2001.0,NaN,1265.0,1794.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/10-Venet...,MARMLS,A11635934,N,Y,25.791150,-80.146649,https://www.redfin.com/FL/Miami-Beach/10-Venet...,None,None,None,None,None,None,None,None,7,blue,THE GRAND VENETIAN CONDO at 10 Venetian Way #1...
7,PAST SALE,2024-11-06,Condo/Co-op,111 Edgewater Dr Unit 2A,Coral Gables,FL,33133.0,1700000.0,4.0,4.0,DAKOTA CONDO,2510.0,NaN,1992.0,NaN,677.0,1111.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/111-Edg...,MARMLS,A11630507,N,Y,25.707219,-80.256084,https://www.redfin.com/FL/Coral-Gables/111-Edg...,None,None,None,None,None,None,None,None,8,blue,DAKOTA CONDO at 111 Edgewater Dr Unit 2A in Co...
8,PAST SALE,2024-11-07,Condo/Co-op,13645 Deering Bay Dr #154,Coral Gables,FL,33158.0,1617946.0,3.0,3.5,DEERING BAY CONDO I,3140.0,NaN,1993.0,NaN,515.0,2894.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/13645-D...,MARMLS,A11628250,N,Y,25.634885,-80.291061,https://w

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: GROVENOR HOUSE CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,400,000M to $5,400,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami-Dade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Sunny Isles Beach, Miami Beach, Key Biscayne, Aventura, Miami, Coral Gables

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 83 condo sales totaling $61,054,385 million from Nov. 3rd to Nov. 10th. The previous week, brokers closed 120 condo sales totaling $92,500,000.

Last week’s units sold for an average of $735,595, lower than the $770,797 average price from the previous week. 

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-05,Condo/Co-op,2627 S Bayshore Dr #1901,Miami,FL,33133.0,5400000.0,5.0,5.5,GROVENOR HOUSE CONDO,4026.0,NaN,2005.0,NaN,1341.0,5859.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11642744,N,Y,25.732211,-80.234936,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,None,None,None,None,None,None,None,None,1,orange,GROVENOR HOUSE CONDO at 2627 S Bayshore Dr #19...
1,PAST SALE,2024-11-05,Condo/Co-op,100 S Pointe Dr #1507,Miami Beach,FL,33139.0,5000000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1870.0,NaN,2002.0,NaN,2674.0,4417.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11501307,N,Y,25.767161,-80.132776,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,None,None,None,None,None,None,None,None,2,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #1...
2,PAST SALE,2024-11-04,Condo/Co-op,201 Crandon Blvd #900,Key Biscayne,FL,33149.0,2550000.0,3.0,3.0,KEY COLONY NO 1,2169.0,NaN,1978.0,NaN,1176.0,2323.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,MARMLS,A11653306,N,Y,25.696814,-80.158094,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,None,None,None,None,None,None,None,None,3,blue,KEY COLONY NO 1 at 201 Crandon Blvd #900 in Ke...
3,PAST SALE,2024-11-05,Condo/Co-op,18201 Collins Ave #4004,Sunny Isles Beach,FL,33160.0,2300000.0,3.0,3.5,TRUMP ROYALE CONDO,2174.0,NaN,2008.0,NaN,1058.0,5235.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11584543,N,Y,25.944933,-80.120001,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,None,None,None,None,None,None,None,None,4,blue,TRUMP ROYALE CONDO at 18201 Collins Ave #4004 ...
4,PAST SALE,2024-11-05,Condo/Co-op,881 Ocean Dr Unit 18E,Key Biscayne,FL,33149.0,1900000.0,3.0,2.0,CASA DEL MAR CONDO,1875.0,NaN,1971.0,NaN,1013.0,1661.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,MARMLS,A11585591,N,Y,25.685659,-80.157201,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,None,None,None,None,None,None,None,None,5,blue,CASA DEL MAR CONDO at 881 Ocean Dr Unit 18E in...
5,PAST SALE,2024-11-06,Condo/Co-op,20201 E Country Club Dr #1105,Aventura,FL,33180.0,1850000.0,3.0,4.0,HAMPTONS SOUTH CONDO,3280.0,NaN,2004.0,NaN,564.0,2800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20201-E-Countr...,MARMLS,A11620826,N,Y,25.964687,-80.125368,https://www.redfin.com/FL/Miami/20201-E-Countr...,None,None,None,None,None,None,None,None,6,blue,HAMPTONS SOUTH CONDO at 20201 E Country Club D...
6,PAST SALE,2024-11-04,Condo/Co-op,10 Venetian Way #1103,Miami Beach,FL,33139.0,1850000.0,2.0,2.5,THE GRAND VENETIAN CONDO,1463.0,NaN,2001.0,NaN,1265.0,1794.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/10-Venet...,MARMLS,A11635934,N,Y,25.791150,-80.146649,https://www.redfin.com/FL/Miami-Beach/10-Venet...,None,None,None,None,None,None,None,None,7,blue,THE GRAND VENETIAN CONDO at 10 Venetian Way #1...
7,PAST SALE,2024-11-06,Condo/Co-op,111 Edgewater Dr Unit 2A,Coral Gables,FL,33133.0,1700000.0,4.0,4.0,DAKOTA CONDO,2510.0,NaN,1992.0,NaN,677.0,1111.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/111-Edg...,MARMLS,A11630507,N,Y,25.707219,-80.256084,https://www.redfin.com/FL/Coral-Gables/111-Edg...,None,None,None,None,None,None,None,None,8,blue,DAKOTA CONDO at 111 Edgewater Dr Unit 2A in Co...
8,PAST SALE,2024-11-07,Condo/Co-op,13645 Deering Bay Dr #154,Coral Gables,FL,33158.0,1617946.0,3.0,3.5,DEERING BAY CONDO I,3140.0,NaN,1993.0,NaN,515.0,2894.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/13645-D...,MARMLS,A11628250,N,Y,25.634885,-80.291061,https://w

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: GROVENOR HOUSE CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,400,000M to $5,400,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami-Dade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Sunny Isles Beach, Miami Beach, Key Biscayne, Aventura, Miami, Coral Gables

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 83 condo sales totaling $61,054,385 million from Nov. 3rd to Nov. 10th. The previous week, brokers closed 120 condo sales totaling $92,500,000.

Last week’s units sold for an average of $735,595, lower than the $770,797 average price from the previous week. 

In [48]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Miami/2627-S-Bayshore-Dr-33133/unit-1901/home/43343001


In [49]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/100-S-Pointe-Dr-33139/unit-1507/home/43282097


In [72]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/3266-Virginia-St-33133/unit-B3/home/42755414


In [74]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Coral-Gables/13645-Deering-Bay-Dr-33158/unit-154/home/42815629


In [75]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/100-S-Pointe-Dr-33139/unit-1507/home/43282097


In [76]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Coral-Gables/13645-Deering-Bay-Dr-33158/unit-154/home/42815629


In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-05,Condo/Co-op,2627 S Bayshore Dr #1901,Miami,FL,33133.0,5400000.0,5.0,5.5,GROVENOR HOUSE CONDO,4026.0,NaN,2005.0,NaN,1341.0,5859.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11642744,N,Y,25.732211,-80.234936,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,None,None,None,None,None,None,None,None,1,orange,GROVENOR HOUSE CONDO at 2627 S Bayshore Dr #19...
1,PAST SALE,2024-11-05,Condo/Co-op,100 S Pointe Dr #1507,Miami Beach,FL,33139.0,5000000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1870.0,NaN,2002.0,NaN,2674.0,4417.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11501307,N,Y,25.767161,-80.132776,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,None,None,None,None,None,None,None,None,2,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #1...
2,PAST SALE,2024-11-04,Condo/Co-op,201 Crandon Blvd #900,Key Biscayne,FL,33149.0,2550000.0,3.0,3.0,KEY COLONY NO 1,2169.0,NaN,1978.0,NaN,1176.0,2323.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,MARMLS,A11653306,N,Y,25.696814,-80.158094,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,None,None,None,None,None,None,None,None,3,blue,KEY COLONY NO 1 at 201 Crandon Blvd #900 in Ke...
3,PAST SALE,2024-11-05,Condo/Co-op,18201 Collins Ave #4004,Sunny Isles Beach,FL,33160.0,2300000.0,3.0,3.5,TRUMP ROYALE CONDO,2174.0,NaN,2008.0,NaN,1058.0,5235.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11584543,N,Y,25.944933,-80.120001,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,None,None,None,None,None,None,None,None,4,blue,TRUMP ROYALE CONDO at 18201 Collins Ave #4004 ...
4,PAST SALE,2024-11-05,Condo/Co-op,881 Ocean Dr Unit 18E,Key Biscayne,FL,33149.0,1900000.0,3.0,2.0,CASA DEL MAR CONDO,1875.0,NaN,1971.0,NaN,1013.0,1661.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,MARMLS,A11585591,N,Y,25.685659,-80.157201,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,None,None,None,None,None,None,None,None,5,blue,CASA DEL MAR CONDO at 881 Ocean Dr Unit 18E in...
5,PAST SALE,2024-11-06,Condo/Co-op,20201 E Country Club Dr #1105,Aventura,FL,33180.0,1850000.0,3.0,4.0,HAMPTONS SOUTH CONDO,3280.0,NaN,2004.0,NaN,564.0,2800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20201-E-Countr...,MARMLS,A11620826,N,Y,25.964687,-80.125368,https://www.redfin.com/FL/Miami/20201-E-Countr...,None,None,None,None,None,None,None,None,6,blue,HAMPTONS SOUTH CONDO at 20201 E Country Club D...
6,PAST SALE,2024-11-04,Condo/Co-op,10 Venetian Way #1103,Miami Beach,FL,33139.0,1850000.0,2.0,2.5,THE GRAND VENETIAN CONDO,1463.0,NaN,2001.0,NaN,1265.0,1794.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/10-Venet...,MARMLS,A11635934,N,Y,25.791150,-80.146649,https://www.redfin.com/FL/Miami-Beach/10-Venet...,None,None,None,None,None,None,None,None,7,blue,THE GRAND VENETIAN CONDO at 10 Venetian Way #1...
7,PAST SALE,2024-11-06,Condo/Co-op,111 Edgewater Dr Unit 2A,Coral Gables,FL,33133.0,1700000.0,4.0,4.0,DAKOTA CONDO,2510.0,NaN,1992.0,NaN,677.0,1111.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/111-Edg...,MARMLS,A11630507,N,Y,25.707219,-80.256084,https://www.redfin.com/FL/Coral-Gables/111-Edg...,None,None,None,None,None,None,None,None,8,blue,DAKOTA CONDO at 111 Edgewater Dr Unit 2A in Co...
8,PAST SALE,2024-11-07,Condo/Co-op,13645 Deering Bay Dr #154,Coral Gables,FL,33158.0,1617946.0,3.0,3.5,DEERING BAY CONDO I,3140.0,NaN,1993.0,NaN,515.0,2894.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/13645-D...,MARMLS,A11628250,N,Y,25.634885,-80.291061,https://w

## Time on Market Calculator

In [77]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 12, 14) ## List (Earlier) date
date2 = datetime(2024, 11, 5) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

327


## Clean CSV for Datawrapper Chart

In [22]:
chart_df = df_top_ten

In [23]:
chart_df = chart_df.fillna(" ")

In [24]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [25]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [30]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
#     "RANK",
#     "COLOR",
#     "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [31]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [32]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [33]:
csv_date_string = today.strftime("%m_%d_%Y")

In [34]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [35]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-11-05,2627 S Bayshore Dr #1901 in Miami,"$5,400,000",5,5.5,"4,026",2005,"$1,341",MARMLS,A11642744,Paola Garcia-Carrillo Residence Realty Inc.,Michael Light Douglas Elliman
1,2024-11-05,100 S Pointe Dr #1507 in Miami Beach,"$5,000,000",2,2.5,"1,870",2002,"$2,674",MARMLS,A11501307,"Aldo Massaglia AG Real Estate Advisors, LLC",Sheila Rojas Brown Harris Stevens Deven Rojas ...
2,2024-11-04,201 Crandon Blvd #900 in Key Biscayne,"$2,550,000",3,3.0,"2,169",1978,"$1,176",MARMLS,A11653306,Carla Datorre Datorre Realty Group,Birgit Lahaye Coldwell Banker Realty
3,2024-11-05,18201 Collins Ave #4004 in Sunny Isles Beach,"$2,300,000",3,3.5,"2,174",2008,"$1,058",MARMLS,A11584543,"Yanina Velez Grand Realty of America, Corp.",Yanine Arabe EXP Realty LLC
4,2024-11-05,881 Ocean Dr Unit 18E in Key Biscayne,"$1,900,000",3,2.0,"1,875",1971,"$1,013",MARMLS,A11585591,"Carole Smith Compass Florida, LLC. Cristina Fo...",Natalia Soria Fortune Christie's International...
5,2024-11-06,20201 E Country Club Dr #1105 in Aventura,"$1,850,000",3,4.0,"3,280",2004,$564,MARMLS,A11620826,"Aimee Kamagi Finvarb Realty, Inc.","Ariel Hollender Compass Florida, LLC."
6,2024-11-04,10 Venetian Way #1103 in Miami Beach,"$1,850,000",2,2.5,"1,463",2001,"$1,265",MARMLS,A11635934,Anna Sherrill One Sotheby's International Re,Marko Gojanovic One Sotheby's International Re...
7,2024-11-06,111 Edgewater Dr Unit 2A in Coral Gables,"$1,700,000",4,4.0,"2,510",1992,$677,MARMLS,A11630507,"Mauricio Barba Compass Florida, LLC.",John Marquez Ocean Max Realty
8,2024-11-07,13645 Deering Bay Dr #154 in Coral Gables,"$1,617,946",3,3.5,"3,140",1993,$515,MARMLS,A11628250,"Adam Levy Compass Florida, LLC. Chris Crystal ...",Olalla Gimenez Gil Great Properties Internatio...
9,2024-11-04,3266 Virginia St Unit B-3 in Miami,"$1,400,000",2,3.0,"1,620",1993,$864,MARMLS,A11659541,Jill Penman One Sotheby's International Realty...,"Anthony Lauria Compass Florida, LLC Riley Smit..."
